# Notebook 02: Generation Capacity EDA

In this one we'll load in the cleaned data and just play around with it to see what we can learn

In [1]:
import pandas as pd
from src.data.load_data import GenerationCapacity
from pandas_profiling import ProfileReport
import altair as alt
import altair_data_server

alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [2]:
df = GenerationCapacity().load_cleaned()

In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36333 entries, 0 to 37278
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   ID                     36333 non-null  int64   
 1   technology             36333 non-null  category
 2   source                 36333 non-null  category
 3   source_type            36333 non-null  category
 4   year                   36333 non-null  int64   
 5   country                36333 non-null  category
 6   capacity_definition    36333 non-null  category
 7   capacity               36333 non-null  float64 
 8   energy_source_level_0  36333 non-null  int64   
 9   energy_source_level_1  36333 non-null  int64   
 10  energy_source_level_2  36333 non-null  int64   
 11  energy_source_level_3  36333 non-null  int64   
 12  technology_level       36333 non-null  int64   
dtypes: category(5), float64(1), int64(7)
memory usage: 2.7 MB


In [4]:
df.sample(10)

,ID,technology,source,source_type,year,country,capacity_definition,capacity,energy_source_level_0,energy_source_level_1,energy_source_level_2,energy_source_level_3,technology_level
35866,2187,Other or unspecified energy sources,ENTSO-E Power Statistics,Other association,2016,EE,Net capacity,0.00,0,1,1,1,1
30050,3450,Renewable energy sources,ENTSO-E Data Portal,Other association,2011,LT,Net capacity,1128.00,0,1,0,0,0
28228,1628,Biomass and biogas,ENTSO-E Data Portal,Other association,2015,PT,Net capacity,613.00,0,0,0,1,1
581,581,Nuclear,Agencija za energijo,Ministry,2013,SI,Net capacity,696.00,0,1,1,1,1
33651,3336,Total,ENTSO-E Transparency Platform,Other association,2015,PL,Net capacity,35572.53,1,0,0,0,0
34051,372,Differently categorized solar,ENTSO-E Power Statistics,Other association,2016,CH,Net capacity,0.00,0,0,0,0,1
28171,1571,Biomass and biogas,ENTSO-E Data Portal,Other association,2011,IT,Net capacity,0.00,0,0,0,1,1
6552,4579,Biomass and biogas,EUROSTAT,Statistical Office,2015,DK,Unknown,105.00,0,0,0,1,1
28420,1820,Fossil fuels,ENTSO-E Data Portal,Other association,2015,NL,Net capacity,25572.00,0,1,0,0,0
27822,1222,Wind,ENTSO-E Data Portal,Other association,2014,RO,Net capacity,2944.00,0,0,1,1,0


So what I want to do with this is:

* Add a % of total capacity column which calculates on each unique combination of `['technology', 'year', 'country']`

* So anything with technology == Total would just be 100%

* But all the other technologies would be a % of that countries total generation for that year

I'm not entirely sure how to do this...